In [82]:

import sys
!{sys.executable} -m pip install -r requirements.txt

DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support
     |████████████████████████████████| 51kB 767kB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/be/49/ae6a7ee2e840017653beff7ed0548d44cac799ccf8e970727ab56f6a6095/clean-text-0.1.1.tar.gz
     |████████████████████████████████| 112kB 465kB/s eta 0:00:01
  Created wheel for ftfy: filename=ftfy-4.4.3-cp27-none-any.whl size=41071 sha256=c3228375f96d39219fd3a7b23c79dbfed1a92a6e7b101dcc6c8c714609577f1a
  Stored in directory: /Users/mac/Library/Caches/pip/wheels/37/54/00/d320239bfc8aad1455314f302dd82a75253fc585e17b81704e
  Created wheel for clean-text: filename=clean_text-0.1.1-cp27-none-any.whl size=8312 sha256=8605603fb934ca

In [83]:
from tqdm import tqdm
import pandas as pd
import pprint
import project_helper

import lxml

from pymongo import MongoClient



In [141]:
cik_lookup = {
    'AMZN': '0001018724',
'BMY': '0000014272',   
    'CNP': '0001130310',
    'CVX': '0000093410',
    'FL': '0000850209',
    'FRT': '0000034903',
    'HON': '0000773840'}

In [142]:
client = MongoClient('localhost', 27017)
client = MongoClient('mongodb://localhost:27017')
db = client.pymongo_test
db = client['pymongo_test']


In [143]:
sec_api = project_helper.SecAPI()


In [144]:
%psource project_helper


In [145]:
from bs4 import BeautifulSoup
import re
import datetime

def get_sec_data(cik, doc_type='',dateb='', owner='include', start=0, count=100,search_text=''):
    newest_pricing_data = pd.to_datetime('2018-01-01')

    
    
    rss_url = 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany' \
        '&CIK={}&type={}&dateb={}&owner={}&count={}&search_text={}&output=atom' \
        .format(cik, doc_type, dateb, owner , start ,count , search_text)
    
    print(rss_url)
    
    
    sec_data = sec_api.get(rss_url)
    feed = BeautifulSoup(sec_data.encode('ascii'), 'html').feed
    entries = [
        (
            
            entry.content.find('filing-date').getText())
        for entry in feed.find_all('entry', recursive=False)
        
        if pd.to_datetime(entry.content.find('filing-date').getText()) >= newest_pricing_data]
    print entries

    return entries
    


In [146]:
example_ticker = 'AMZN'
sec_data = {}


for ticker, cik in cik_lookup.items():
    sec_data[ticker] = get_sec_data(cik)

pprint.pprint(sec_data[example_ticker][:5])

https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=0001130310&type=&dateb=&owner=include&count=0&search_text=100&output=atom
[u'2020-08-03', u'2020-07-31', u'2020-07-31', u'2020-07-30', u'2020-07-30', u'2020-07-30', u'2020-07-20', u'2020-07-06', u'2020-07-06', u'2020-07-06']
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=0000093410&type=&dateb=&owner=include&count=0&search_text=100&output=atom
[u'2020-08-05', u'2020-07-31', u'2020-07-31', u'2020-07-31', u'2020-07-22', u'2020-07-20', u'2020-07-20', u'2020-07-20', u'2020-07-20', u'2020-07-20']
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=0000773840&type=&dateb=&owner=include&count=0&search_text=100&output=atom
[u'2020-08-05', u'2020-08-04', u'2020-08-03', u'2020-08-03', u'2020-08-03', u'2020-07-30', u'2020-07-29', u'2020-07-28', u'2020-07-28', u'2020-07-24']
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=0000034903&type=&dateb=&owner=include&count=0&search_text=100&output=atom

In [147]:
sec_data

{'AMZN': [u'2020-08-05',
  u'2020-08-05',
  u'2020-08-05',
  u'2020-08-05',
  u'2020-08-05',
  u'2020-07-31',
  u'2020-07-31',
  u'2020-07-30',
  u'2020-07-02',
  u'2020-07-02'],
 'BMY': [u'2020-08-04',
  u'2020-07-30',
  u'2020-07-16',
  u'2020-07-16',
  u'2020-07-13',
  u'2020-07-06',
  u'2020-07-06',
  u'2020-07-02',
  u'2020-07-02',
  u'2020-07-02'],
 'CNP': [u'2020-08-03',
  u'2020-07-31',
  u'2020-07-31',
  u'2020-07-30',
  u'2020-07-30',
  u'2020-07-30',
  u'2020-07-20',
  u'2020-07-06',
  u'2020-07-06',
  u'2020-07-06'],
 'CVX': [u'2020-08-05',
  u'2020-07-31',
  u'2020-07-31',
  u'2020-07-31',
  u'2020-07-22',
  u'2020-07-20',
  u'2020-07-20',
  u'2020-07-20',
  u'2020-07-20',
  u'2020-07-20'],
 'FL': [u'2020-08-03',
  u'2020-08-03',
  u'2020-07-23',
  u'2020-07-16',
  u'2020-07-06',
  u'2020-07-06',
  u'2020-07-06',
  u'2020-07-06',
  u'2020-07-06',
  u'2020-07-06'],
 'FRT': [u'2020-08-05',
  u'2020-08-05',
  u'2020-06-22',
  u'2020-05-12',
  u'2020-05-11',
  u'2020-05-11',
 

In [47]:
raw_fillings_by_ticker = {}

for ticker, data in sec_data.items():
    raw_fillings_by_ticker[ticker] = {}
    for index_url, file_type, file_date in tqdm(data, desc='Downloading {} Fillings'.format(ticker), unit='filling'):
        if (file_type == '10-K'):
            file_url = index_url.replace('-index.htm', '.txt').replace('.txtl', '.txt')            
            
            raw_fillings_by_ticker[ticker][file_date] = sec_api.get(file_url)


print('Example Document:\n\n{}...'.format(next(iter(raw_fillings_by_ticker[example_ticker].values()))[:1000]))

Example Document:

-----BEGIN PRIVACY-ENHANCED MESSAGE-----
Proc-Type: 2001,MIC-CLEAR
Originator-Name: webmaster@www.sec.gov
Originator-Key-Asymmetric:
 MFgwCgYEVQgBAQICAf8DSgAwRwJAW2sNKK9AVtBzYZmr6aGjlWyK3XmZv3dTINen
 TWSM7vrzLADbmYQaionwg5sDW3P6oaM5D3tdezXMm7z1T+B+twIDAQAB
MIC-Info: RSA-MD5,RSA,
 JheN9rrH0LADcPLNpdgic2Y+wa34llXqe750TowVzUFLd6IuuNQne2H0axAosYFz
 1Lmg9dJfEMxBnv2E6NCBrw==

<SEC-DOCUMENT>0001193125-05-047032.txt : 20050311
<SEC-HEADER>0001193125-05-047032.hdr.sgml : 20050311
<ACCEPTANCE-DATETIME>20050311062027
ACCESSION NUMBER:		0001193125-05-047032
CONFORMED SUBMISSION TYPE:	10-K
PUBLIC DOCUMENT COUNT:		9
CONFORMED PERIOD OF REPORT:	20041231
FILED AS OF DATE:		20050311
DATE AS OF CHANGE:		20050311

FILER:

	COMPANY DATA:	
		COMPANY CONFORMED NAME:			AMAZON COM INC
		CENTRAL INDEX KEY:			0001018724
		STANDARD INDUSTRIAL CLASSIFICATION:	RETAIL-CATALOG & MAIL-ORDER HOUSES [5961]
		IRS NUMBER:				911646860
		STATE OF INCORPORATION:			DE
		FISCAL YEAR END:			1231

	FILING VA